In [1]:
# ADDED:
model_path = "meta-llama/Llama-2-7b-chat-hf" # not enough RAM or disk space for larger models

In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer,AutoModelForCausalLM

# change device map from "auto" to None when using CPU
tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=None)

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()
#

# change device map from "auto" to None when using CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=None)

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()

# move to CPU (not enough GPU space)
device = torch.device("cpu")
model.to(device)
#

/Users/christinearnoldt/mambaforge/envs/SelfIE/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/christinearnoldt/mambaforge/envs/SelfIE/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [3]:
from selfie.interpret import InterpretationPrompt, interpret
interpretation_prompt = InterpretationPrompt(tokenizer, ("[INST]", 0, 0, 0, 0, 0, "[/INST] Sure, I will summarize the message:"))

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()
#

[1, 518, 25580, 29962] 4
[1, 518, 25580, 21540, 29871] 5
[1, 518, 25580, 21540, 903, 29871] 6
[1, 518, 25580, 21540, 903, 903, 29871] 7
[1, 518, 25580, 21540, 903, 903, 903, 29871] 8


In [7]:
original_prompt = "[INST] What's highest mountain in the world? [/INST]"
tokens_to_interpret = [(10,7), (10,8)]
bs = 16 #originally: 2
max_new_tokens = 20
k = 5 #originally: 1

interpretation_df = interpret(original_prompt=original_prompt, tokens_to_interpret=tokens_to_interpret, model=model, interpretation_prompt=interpretation_prompt, bs=bs, max_new_tokens=max_new_tokens, k=k, tokenizer=tokenizer)

Interpreting '[INST] What's highest mountain in the world? [/INST]' with '[INST]_ _ _ _ _ [/INST] Sure, I will summarize the message:'
all_insert_infos [{'replacing_mode': 'normalized', 'overlay_strength': 1, 'retrieve_layer': 10, 'retrieve_token': 7, 5: ([4, 5, 6, 7, 8], tensor([[[-0.1717,  0.0836, -0.0956,  ...,  0.0559,  0.0541,  0.1325],
         [-0.1717,  0.0836, -0.0956,  ...,  0.0559,  0.0541,  0.1325],
         [-0.1717,  0.0836, -0.0956,  ...,  0.0559,  0.0541,  0.1325],
         [-0.1717,  0.0836, -0.0956,  ...,  0.0559,  0.0541,  0.1325],
         [-0.1717,  0.0836, -0.0956,  ...,  0.0559,  0.0541,  0.1325]]],
       grad_fn=<RepeatBackward0>))}, {'replacing_mode': 'normalized', 'overlay_strength': 1, 'retrieve_layer': 10, 'retrieve_token': 8, 5: ([4, 5, 6, 7, 8], tensor([[[-0.2543,  0.3470, -0.1114,  ...,  0.0471, -0.3432,  0.0043],
         [-0.2543,  0.3470, -0.1114,  ...,  0.0471, -0.3432,  0.0043],
         [-0.2543,  0.3470, -0.1114,  ...,  0.0471, -0.3432,  0.0043],


100%|██████████| 1/1 [15:16<00:00, 916.94s/it]


In [8]:
pd.DataFrame(interpretation_df)

,prompt,interpretation,layer,token,token_decoded
0,[INST] What's highest mountain in the world? [...,"\n\nThe message is a series of underscores, wh...",10,7,highest
1,[INST] What's highest mountain in the world? [...,\n\nThe message appears to be a series of rand...,10,8,mountain


In [9]:
interpretation_df

{'prompt': ["[INST] What's highest mountain in the world? [/INST]",
  "[INST] What's highest mountain in the world? [/INST]"],
 'interpretation': ['\n\nThe message is a series of underscores, which typically indicates that the message is incomplete',
  '\n\nThe message appears to be a series of random characters, and there is no clear meaning or'],
 'layer': [10, 10],
 'token': [7, 8],
 'token_decoded': ['highest', 'mountain']}